In [126]:
import os;
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer;
import pandas as pd;
import arcpy;


##### Create the output location using arcpy
I assume you are using your ArcGIS Pro version of Jupyter and thus have arcpy

In [131]:
out_path = r"./data";
out_fgdb = "michelle.gdb";
out_fc   = "baltimore";

if not os.path.exists(out_path):
    os.mkdir(out_path);
    
if not arcpy.Exists(out_path + os.sep + out_fgdb):
    arcpy.CreateFileGDB_management(out_path,out_fgdb);
    

##### Get the AGO item to export


gis = GIS();

data_item_id = "48416818f88b4be2b0f4ae51c7898341";
data_item = gis.content.get(data_item_id);

data_item


##### Sniff out the feature service from the layer listing

I am not sure why Baltimore is using http here, need to flip to https.

In [135]:
feature_layer_url = data_item.get_data()['operationalLayers'][0]['url'];

feature_layer_url = feature_layer_url.replace('http://','https://');

feature_layer_url


'https://geodata.baltimorecity.gov/egis/rest/services/Environmental_Police/Collector/FeatureServer/0'

##### Pull down the records into the sdf

This should iterate as needed to circumvent the record limit on the service

In [136]:
feature_layer = FeatureLayer((feature_layer_url));

sdf = feature_layer.query(as_df=True);

len(sdf.index)


60

##### Plot the records on the map

And this does not work as Python API is super buggy.  If it works then they have fixed it.

In [137]:
map0 = GIS().map("Baltimore");

sdf.spatial.plot(map0);

map0


MapView(layout=Layout(height='400px', width='100%'))

##### Export the records to a file geodatabase feature class

Hey, at least this works

In [138]:
sdf.spatial.to_featureclass(out_path + os.sep + out_fgdb + os.sep + out_fc);
